# Korrelations- und Verteilungsanalyse: Körpergröße, Schuhgröße und Gewicht

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_04/korrelationsanalyse_beispiel.ipynb)

**Lernziele:**
- Normalverteilung erkennen und fitten
- Scatterplot und linearer Fit
- Korrelationen berechnen
- Statistische Tests und ihre Tücken

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Styling
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

np.random.seed(42)
print("📊 Bibliotheken erfolgreich importiert!")

## 1. Datenerstellung

In [ ]:
# Explizite Werte für die drei Listen
#koerpergroesse = [172, 180, 168, 175, 182, 177, 169, 173, 178, 165, 170, 176, 181, 174, 179, 167, 171, 183, 166, 177, 172, ]
#schuhgroesse = [42, 44, 40, 43, 45, 44, 41, 42, 44, 39, 41, 43, 45, 42, 44, 40, 41, 46, 39, 43, 42]
#gewicht = [68, 85, 62, 75, 90, 80, 65, 70, 78, 60, 66, 77, 88, 72, 82, 63, 69, 92, 61, 79, 71]

koerpergroesse = [188, 194, 178, 179, 165, 163, 172, 183, 181, 195, 186, 196, 179 , 190, 187, 165, 190, 169, 183, 177, 170]
gewicht = [108, 91, 60, 84, 70, 58, 72, 80, 70, 90, 93, 96, 95, 83, 99, 55, 85, 62, 84, 77, 70]
schuhgroesse = [45, 43, 42, 44, 39, 39, 42, 43, 43, 47, 45, 47, 46, 46, 44, 41, 47, 39, 44, 42, 40]
#gewicht = [108, 91, 60, 84, 70, 58, 72, 80, 70, 90, 93, 96, 95, 83, 99, 100, 55, 85, 62, 84, 77, 70]


df = pd.DataFrame({
    'Körpergröße [cm]': koerpergroesse,
    'Schuhgröße [EU]': schuhgroesse,
    'Gewicht [kg]': gewicht
})
print("=== DATEN (21 Personen) ===")
print(df.head(10))
print(f'Deskriptive Statistik:')
print(df.describe().round(2))

## 2. Histogramm mit Normalverteilungs-Fit

Wir wählen die **Körpergröße** und überprüfen, ob sie normalverteilt ist.

In [ ]:
variable = 'Körpergröße [cm]'; data = df[variable].values
mu, sigma = stats.norm.fit(data)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
n_bins = 16
counts, bins, patches = ax.hist(data, bins=n_bins, density=True, alpha=0.7, color='skyblue', edgecolor='black', linewidth=1)
x_fit = np.linspace(data.min() - 5, data.max() + 5, 100)
y_fit = stats.norm.pdf(x_fit, mu, sigma)
ax.plot(x_fit, y_fit, 'r-', linewidth=3, label=f'Normalverteilung\nμ = {mu:.1f}, σ = {sigma:.1f}')
ax.set_xlabel(variable, fontsize=12)
ax.set_ylabel('Dichte', fontsize=12)
ax.set_title(f'Histogramm: {variable} mit Normalverteilungs-Fit', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
shapiro_stat, shapiro_p = stats.shapiro(data)
ax.text(0.02, 0.98, f'Shapiro-Wilk Test:\nW-Wert = {shapiro_stat:.3f}\np-Wert = {shapiro_p:.3f}', transform=ax.transAxes, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor="wheat", alpha=0.8))
plt.tight_layout()
plt.show()
print(f'📊 Normalverteilungstest für {variable}:')
print(f'   Geschätzte Parameter: μ = {mu:.2f}, σ = {sigma:.2f}')
print(f'   Shapiro-Wilk p-Wert: {shapiro_p:.3f}')
if shapiro_p > 0.05:
    print('   ✅ Normalverteilung wird nicht abgelehnt (p > 0.05)')
else:
    print('   ⚠️  Daten weichen von Normalverteilung ab (p ≤ 0.05)')

**Wie interpretiert man das Ergebnis des Shapiro-Wilk-Tests?**

Der Shapiro-Wilk-Test prüft, ob die Daten normalverteilt sind.
- Die sogenannte *Nullhypothese* lautet: Die Daten sind normalverteilt.
- Der Test berechnet eine **Teststatistik** und einen **p-Wert**.

**Die Teststatistik (W-Wert):**
- Liegt immer zwischen 0 und 1
- Je näher der Wert bei 1, desto besser passen die Daten zu einer Normalverteilung
- Ein niedriger Wert (z.B. 0.7) zeigt starke Abweichung von der Normalverteilung
- Ein hoher Wert (z.B. 0.98) zeigt gute Übereinstimmung mit der Normalverteilung

**Der p-Wert:**
- Ist der p-Wert **größer als 0.05**, gibt es keinen Hinweis auf eine Abweichung von der Normalverteilung. Die Daten *könnten* also normalverteilt sein.
- Ist der p-Wert **kleiner oder gleich 0.05**, spricht das gegen Normalverteilung. Die Daten sind dann mit hoher Wahrscheinlichkeit *nicht* normalverteilt.

**Achtung:**
Auch wenn der p-Wert groß ist, heißt das nicht, dass die Daten garantiert normalverteilt sind – nur, dass man es mit den vorliegenden Daten nicht widerlegen kann.

**Tipp:**
Schau dir immer auch das Histogramm an! Test und Grafik zusammen geben das beste Bild.

## 3. Scatterplot mit linearem Fit

Wir untersuchen den Zusammenhang zwischen **Körpergröße** und **Gewicht**.

In [ ]:
var_x = 'Körpergröße [cm]'; var_y = 'Gewicht [kg]'
x_data = df[var_x].values; y_data = df[var_y].values
slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, y_data)
fig, ax = plt.subplots(1, 1, figsize=(10, 7))
ax.scatter(x_data, y_data, s=100, alpha=0.7, color='blue', edgecolors='black', linewidth=1)
x_fit = np.linspace(x_data.min() - 2, x_data.max() + 2, 100)
y_fit = slope * x_fit + intercept
ax.plot(x_fit, y_fit, 'r-', linewidth=2, label=f'y = {slope:.2f}x + {intercept:.1f}')
ax.set_xlabel(var_x, fontsize=12)
ax.set_ylabel(var_y, fontsize=12)
ax.set_title(f'Scatterplot: {var_x} vs. {var_y}', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
r_squared = r_value**2
ax.text(0.02, 0.98, f'Lineare Regression:\nSteigung = {slope:.2f}\nR² = {r_squared:.3f}\nKorrelation r = {r_value:.3f}\np-Wert = {p_value:.3f}', transform=ax.transAxes, verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
plt.tight_layout()
plt.show()
print(f'📈 Lineare Regression: {var_x} → {var_y}')
print(f'   Steigung: {slope:.3f} {var_y.split("[")[1][:-1]} pro {var_x.split("[")[1][:-1]}')
print(f'   Y-Achsenabschnitt: {intercept:.2f} {var_y.split("[")[1][:-1]}')
print(f'   Korrelationskoeffizient r: {r_value:.3f}')
print(f'   Bestimmtheitsmaß R²: {r_squared:.3f} ({r_squared*100:.1f}% der Varianz erklärt)')
signifikanz = 'signifikant' if p_value < 0.05 else 'nicht signifikant'
print(f'   p-Wert: {p_value:.3f} ({signifikanz} bei α=0.05)')

**Was bedeuten die Werte aus `stats.linregress`?**

- **slope**: Die Steigung der Regressionsgeraden. Sie gibt an, um wie viel sich die Zielvariable (y) im Mittel ändert, wenn die erklärende Variable (x) um 1 steigt.
- **intercept**: Der y-Achsenabschnitt der Regressionsgeraden.
- **r_value**: Der Korrelationskoeffizient (r, Pearson). Er misst die Stärke und Richtung des linearen Zusammenhangs zwischen x und y. Werte nahe +1 oder -1 bedeuten einen starken Zusammenhang, Werte nahe 0 keinen Zusammenhang.
- **p_value**: Der p-Wert zum Test, ob die Steigung (slope) signifikant von 0 verschieden ist.
    - **Achtung:** Dieser p-Wert prüft NICHT auf Normalverteilung, sondern ob ein linearer Zusammenhang zwischen x und y statistisch nachweisbar ist.
    - **α (Alpha)**: Das Signifikanzniveau ist ein fester Schwellenwert (meist 0.05 = 5%), den wir vorher festlegen. Der p-Wert wird mit diesem Alpha verglichen, um zu entscheiden, ob das Ergebnis statistisch signifikant ist.
- **std_err**: Der Standardfehler der Steigung. Er gibt an, wie stark die geschätzte Steigung in verschiedenen Stichproben schwanken würde.

**Zusammengefasst:**
- Der p-Wert aus `linregress` ist ein Maß für die Signifikanz des linearen Zusammenhangs, nicht für Normalverteilung!
- Der p-Wert aus dem Shapiro-Wilk-Test prüft auf Normalverteilung.
- α (Alpha) ist das Signifikanzniveau - ein Schwellenwert, mit dem der p-Wert verglichen wird.

**Was bedeutet der p-Wert genau?**

Der p-Wert ist die Wahrscheinlichkeit, unter der Annahme, dass die Nullhypothese wahr ist, **mindestens so extreme Daten** zu beobachten wie die tatsächlich gemessenen.

**Bei der linearen Regression:**
- **Nullhypothese:** Es gibt keinen linearen Zusammenhang (Steigung = 0)
- **p = 0.01 bedeutet:** Wenn es wirklich keinen linearen Zusammenhang gäbe, wäre die Wahrscheinlichkeit nur 1%, eine mindestens so große Steigung zu beobachten
- **Interpretation:** Da 0.01 < 0.05 → statistisch signifikanter linearer Zusammenhang

**Bei Shapiro-Wilk:**
- **Nullhypothese:** Die Daten sind normalverteilt
- **p = 0.01 bedeutet:** Wenn die Daten normalverteilt wären, wäre die Wahrscheinlichkeit nur 1%, eine mindestens so große Abweichung zu sehen
- **Interpretation:** Da 0.01 < 0.05 → Daten sind nicht normalverteilt

**Wichtig:** Der p-Wert ist NICHT die Wahrscheinlichkeit, dass die Nullhypothese wahr ist!

**Warum sind die Nullhypothesen "inkonsistent"?**

Die Nullhypothesen verschiedener Tests sind nicht einheitlich formuliert:

**Shapiro-Wilk-Test:**
- H₀: Die Daten **sind** normalverteilt
- H₁: Die Daten sind **nicht** normalverteilt
- → Kleiner p-Wert = **NICHT** normalverteilt

**Lineare Regression:**
- H₀: Es gibt **keinen** linearen Zusammenhang (Steigung = 0)
- H₁: Es gibt **einen** linearen Zusammenhang (Steigung ≠ 0)
- → Kleiner p-Wert = **GIBT** einen Zusammenhang

**Grund:** Bei Shapiro-Wilk testet man auf eine spezifische Eigenschaft (Normalverteilung), bei der Regression traditionell auf "Kein Effekt". Deshalb ist es wichtig, immer genau zu verstehen, was die Nullhypothese eines Tests besagt!

## 4. Korrelationskoeffizient mit Python-Funktion

Berechnung der Korrelation zwischen **Körpergröße** und **Schuhgröße** mit `numpy` und `scipy`.

In [ ]:
var1 = 'Körpergröße [cm]'; var2 = 'Schuhgröße [EU]'
data1 = df[var1].values; data2 = df[var2].values
numpy_corr = np.corrcoef(data1, data2)[0, 1]
scipy_corr, scipy_p = stats.pearsonr(data1, data2)
pandas_corr = df[[var1, var2]].corr().iloc[0, 1]
spearman_corr, spearman_p = stats.spearmanr(data1, data2)
print(f'📊 Korrelation zwischen {var1} und {var2}:')
print(f'   NumPy corrcoef():     r = {numpy_corr:.4f}')
print(f'   SciPy pearsonr():     r = {scipy_corr:.4f}, p = {scipy_p:.4f}')
print(f'   Pandas corr():        r = {pandas_corr:.4f}')
print(f'   Spearman-Korrelation: ρ = {spearman_corr:.4f}, p = {spearman_p:.4f}')

## 5. Korrelationskoeffizient explizit berechnen

Schritt-für-Schritt Berechnung nach der Definition: $r = \frac{\sum{(x_i - \bar{x})(y_i - \bar{y})}}{\sqrt{\sum{(x_i - \bar{x})^2} \sum{(y_i - \bar{y})^2}}}$

In [ ]:
x = data1; y = data2
x_mean = np.mean(x); y_mean = np.mean(y)
x_diff = x - x_mean; y_diff = y - y_mean
xy_products = x_diff * y_diff; sum_xy_products = np.sum(xy_products)
x_squared = x_diff**2; y_squared = y_diff**2
sum_x_squared = np.sum(x_squared); sum_y_squared = np.sum(y_squared)
denominator = np.sqrt(sum_x_squared * sum_y_squared)
r_manual = sum_xy_products / denominator
print(f'Formel: r = Σ(xi - x̄)(yi - ȳ) / √[Σ(xi - x̄)² × Σ(yi - ȳ)²]')
print(f'Manual: r = {r_manual:.4f}')
print(f'NumPy:  r = {numpy_corr:.4f}')